In [1]:
from keras_vggface.utils import preprocess_input
from project.dg_face import utils, detection, alignment, recognition
import os
import numpy as np

C:\Users\danil\Desktop\neuro\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\danil\Desktop\neuro\venv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\danil\Desktop\neuro\venv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Как будем строить датасет.
* 50 различных сотрудников в компании (можно начать с 10 для картиночки)
* у каждого сотрудника по 3-5 фотографий в тренировочном датасете
* у каждого сорудника по 1(сомнительно) фотографии в тестовом датасете

### Вне компании
* 10 человек вне компании
* их фотографий в тренировочном датасете нет
* они в тестовом датасете


In [2]:
model = recognition.make_cnn_model()
def get_embeddings(base_path):
    X, y = [], []
    for person in os.listdir(base_path):
        for person_img in os.listdir(base_path + "/" + person):
            path = base_path + '/' + person + '/' + person_img
            print(path)
            img = utils.load_image(path)
            faces = detection.detect_faces(img)
            if len(faces) != 1:
                print(path, 'was skipped because of len(faces)=', len(faces))
                continue
            face_image = alignment.aligned_face(img, faces[0])
            face_image = preprocess_input(face_image.astype(float), version=1)
            encoding = model.predict(np.array([face_image]))[0]
            X.append(encoding)
            y.append(person)
    return X, y

In [3]:
X_train, y_train = get_embeddings("images_30_20/train")
X_test, y_test = get_embeddings("images_30_20/test")
print(len(X_train), len(X_test))

images_30_20/train/worker0/001257.jpg
1/1 [==============================] - 0s 312ms/step
images_30_20/train/worker0/008392.jpg
1/1 [==============================] - 0s 141ms/step
images_30_20/train/worker0/013825.jpg
1/1 [==============================] - 0s 172ms/step
images_30_20/train/worker1/185949.jpg
1/1 [==============================] - 0s 219ms/step
images_30_20/train/worker1/187455.jpg
1/1 [==============================] - 0s 141ms/step
images_30_20/train/worker1/194128.jpg
1/1 [==============================] - 0s 156ms/step
images_30_20/train/worker10/165300.jpg
1/1 [==============================] - 0s 141ms/step
images_30_20/train/worker10/168172.jpg
1/1 [==============================] - 0s 141ms/step
images_30_20/train/worker10/173174.jpg
1/1 [==============================] - 0s 141ms/step
images_30_20/train/worker11/186581.jpg
1/1 [==============================] - 0s 141ms/step
images_30_20/train/worker11/191573.jpg
1/1 [==============================] - 0s 141ms

In [5]:
X_train

[array([1.09181297e+00, 4.89941418e-01, 4.87054396e+00, 8.48514557e+00,
        1.68098405e-01, 6.39197767e-01, 4.32422161e+00, 8.29990578e+00,
        6.71611929e+00, 1.16669834e+00, 8.95336390e-01, 1.15266848e+00,
        0.00000000e+00, 2.72124600e+00, 1.28249960e+01, 6.22491884e+00,
        1.21599793e+00, 7.42653942e+00, 1.27063274e+00, 1.16136026e+00,
        1.78652108e+00, 9.24213231e-01, 6.64846182e+00, 8.25346589e-01,
        6.47052860e+00, 7.70890832e-01, 1.51465058e+00, 1.06861115e+00,
        1.50542843e+00, 3.48636675e+00, 5.27645683e+00, 2.35441685e+00,
        1.25564623e+00, 6.07424319e-01, 6.40696001e+00, 3.41352415e+00,
        0.00000000e+00, 2.97211432e+00, 0.00000000e+00, 1.83396912e+00,
        8.00803304e-01, 6.98386848e-01, 4.11538744e+00, 4.19698524e+00,
        6.71591759e-01, 2.01399863e-01, 2.32373977e+00, 1.10023558e-01,
        2.77346253e-01, 2.92653680e+00, 4.05642939e+00, 1.46862566e-01,
        1.92961484e-01, 2.32388042e-02, 1.13168967e+00, 3.476255

In [33]:
np.savez_compressed("npz_30_20/train", X=np.array(X_train), y=np.array(y_train))
np.savez_compressed("npz_30_20/test", X=np.array(X_test), y=np.array(y_test))